In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow import keras
from tensorflow.keras import layers as L
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [2]:
# Чтение DataFrame
df_models = pd.read_csv("данные для моделей.csv")
print("DataFrame 'данные для моделей.csv' успешно загружен.")

DataFrame 'данные для моделей.csv' успешно загружен.


In [3]:
print("\n--- Классификация: превышает ли значение SI медианное значение выборки ---")

# 1. Определение целевой переменной для SI
target_si_median_class = 'si_above_median' # Новая целевая колонка

# Вычисляем медиану для 'log_SI'
median_si = df_models['log_SI'].median()
# Создаем бинарный столбец: 1, если log_SI > медианы, иначе 0
df_models[target_si_median_class] = (df_models['log_SI'] > median_si).astype(int)

print(f"Медианное значение log_SI: {median_si:.4f}")
print(f"Распределение классов для {target_si_median_class}:\n{df_models[target_si_median_class].value_counts()}")

# Признаки (X) и целевая переменная (y)
# Исключаем только исходные логарифмированные целевые переменные и только созданную бинарную целевую переменную для SI.
X = df_models.drop(columns=['log_IC50, mM', 'log_CC50, mM', 'log_SI', target_si_median_class])
y = df_models[target_si_median_class]

# Разделение на обучающую и тестовую выборки
X_train_si_class, X_test_si_class, y_train_si_class, y_test_si_class = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y # stratify=y для сохранения пропорций классов
)

print(f"\nРазмер обучающей выборки (X_train_si_class): {X_train_si_class.shape}")
print(f"Размер тестовой выборки (X_test_si_class): {X_test_si_class.shape}")
print(f"Распределение классов в обучающей выборке:\n{y_train_si_class.value_counts(normalize=True)}")
print(f"Распределение классов в тестовой выборке:\n{y_test_si_class.value_counts(normalize=True)}")


--- Классификация: превышает ли значение SI медианное значение выборки ---
Медианное значение log_SI: 0.5940
Распределение классов для si_above_median:
si_above_median
0    484
1    483
Name: count, dtype: int64

Размер обучающей выборки (X_train_si_class): (773, 68)
Размер тестовой выборки (X_test_si_class): (194, 68)
Распределение классов в обучающей выборке:
si_above_median
0    0.500647
1    0.499353
Name: proportion, dtype: float64
Распределение классов в тестовой выборке:
si_above_median
0    0.5
1    0.5
Name: proportion, dtype: float64


In [4]:
# Метрики для GridSearchCV (остаются теми же)
scoring_metrics_class = {
    'Accuracy': 'accuracy',
    'F1': 'f1',
    'ROC_AUC': 'roc_auc'
}

In [5]:
# Модель 1: Логистическая Регрессия 
print("\n===== Логистическая Регрессия для SI (классификация) =====")
pipeline_lr_class_si = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, solver='liblinear'))
])

param_grid_lr_class_si = {
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l1', 'l2']
}

grid_search_lr_class_si = GridSearchCV(
    pipeline_lr_class_si,
    param_grid_lr_class_si,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_lr_class_si.fit(X_train_si_class, y_train_si_class)

print("Лучшие параметры для Логистической Регрессии:", grid_search_lr_class_si.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_lr_class_si.best_score_)

# Оценка на тестовой выборке
y_pred_lr_class_si = grid_search_lr_class_si.predict(X_test_si_class)
y_proba_lr_class_si = grid_search_lr_class_si.predict_proba(X_test_si_class)[:, 1]

accuracy_lr_class_si = accuracy_score(y_test_si_class, y_pred_lr_class_si)
f1_lr_class_si = f1_score(y_test_si_class, y_pred_lr_class_si)
roc_auc_lr_class_si = roc_auc_score(y_test_si_class, y_proba_lr_class_si)

print("\nМетрики на тестовой выборке (Логистическая Регрессия):")
print(f"Accuracy: {accuracy_lr_class_si:.4f}")
print(f"F1-score: {f1_lr_class_si:.4f}")
print(f"ROC AUC: {roc_auc_lr_class_si:.4f}")
print("\nClassification Report (Логистическая Регрессия):")
print(classification_report(y_test_si_class, y_pred_lr_class_si))


===== Логистическая Регрессия для SI (классификация) =====
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшие параметры для Логистической Регрессии: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Лучший ROC_AUC на кросс-валидации: 0.6741209007442773

Метрики на тестовой выборке (Логистическая Регрессия):
Accuracy: 0.6701
F1-score: 0.6484
ROC AUC: 0.7272

Classification Report (Логистическая Регрессия):
              precision    recall  f1-score   support

           0       0.65      0.73      0.69        97
           1       0.69      0.61      0.65        97

    accuracy                           0.67       194
   macro avg       0.67      0.67      0.67       194
weighted avg       0.67      0.67      0.67       194



In [6]:
#Модель 2: Random Forest Классификатор 
print("\n===== Random Forest Классификатор для SI (классификация) =====")
pipeline_rf_class_si = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

param_grid_rf_class_si = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_features': [0.6, 0.8, 1.0],
    'classifier__min_samples_leaf': [5, 10]
}

grid_search_rf_class_si = GridSearchCV(
    pipeline_rf_class_si,
    param_grid_rf_class_si,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_rf_class_si.fit(X_train_si_class, y_train_si_class)

print("Лучшие параметры для Random Forest:", grid_search_rf_class_si.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_rf_class_si.best_score_)

# Оценка на тестовой выборке
y_pred_rf_class_si = grid_search_rf_class_si.predict(X_test_si_class)
y_proba_rf_class_si = grid_search_rf_class_si.predict_proba(X_test_si_class)[:, 1]

accuracy_rf_class_si = accuracy_score(y_test_si_class, y_pred_rf_class_si)
f1_rf_class_si = f1_score(y_test_si_class, y_pred_rf_class_si)
roc_auc_rf_class_si = roc_auc_score(y_test_si_class, y_proba_rf_class_si)

print("\nМетрики на тестовой выборке (Random Forest):")
print(f"Accuracy: {accuracy_rf_class_si:.4f}")
print(f"F1-score: {f1_rf_class_si:.4f}")
print(f"ROC AUC: {roc_auc_rf_class_si:.4f}")
print("\nClassification Report (Random Forest):")
print(classification_report(y_test_si_class, y_pred_rf_class_si))


===== Random Forest Классификатор для SI (классификация) =====
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Лучшие параметры для Random Forest: {'classifier__max_features': 1.0, 'classifier__min_samples_leaf': 5, 'classifier__n_estimators': 200}
Лучший ROC_AUC на кросс-валидации: 0.7222195553364384

Метрики на тестовой выборке (Random Forest):
Accuracy: 0.6237
F1-score: 0.6138
ROC AUC: 0.6771

Classification Report (Random Forest):
              precision    recall  f1-score   support

           0       0.62      0.65      0.63        97
           1       0.63      0.60      0.61        97

    accuracy                           0.62       194
   macro avg       0.62      0.62      0.62       194
weighted avg       0.62      0.62      0.62       194



In [7]:
# Модель 3: LightGBM Классификатор 
print("\n===== LightGBM Классификатор для SI (классификация) =====")
pipeline_lgbm_class_si = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LGBMClassifier(random_state=42, verbose=-1))
])

param_grid_lgbm_class_si = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__num_leaves': [20, 31],
    'classifier__reg_alpha': [0.1, 0.5],
    'classifier__reg_lambda': [0.1, 0.5]
}

grid_search_lgbm_class_si = GridSearchCV(
    pipeline_lgbm_class_si,
    param_grid_lgbm_class_si,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_lgbm_class_si.fit(X_train_si_class, y_train_si_class)

print("Лучшие параметры для LightGBM:", grid_search_lgbm_class_si.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_lgbm_class_si.best_score_)

# Оценка на тестовой выборке
y_pred_lgbm_class_si = grid_search_lgbm_class_si.predict(X_test_si_class)
y_proba_lgbm_class_si = grid_search_lgbm_class_si.predict_proba(X_test_si_class)[:, 1]

accuracy_lgbm_class_si = accuracy_score(y_test_si_class, y_pred_lgbm_class_si)
f1_lgbm_class_si = f1_score(y_test_si_class, y_pred_lgbm_class_si)
roc_auc_lgbm_class_si = roc_auc_score(y_test_si_class, y_proba_lgbm_class_si)

print("\nМетрики на тестовой выборке (LightGBM):")
print(f"Accuracy: {accuracy_lgbm_class_si:.4f}")
print(f"F1-score: {f1_lgbm_class_si:.4f}")
print(f"ROC AUC: {roc_auc_lgbm_class_si:.4f}")
print("\nClassification Report (LightGBM):")
print(classification_report(y_test_si_class, y_pred_lgbm_class_si))


===== LightGBM Классификатор для SI (классификация) =====
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Лучшие параметры для LightGBM: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100, 'classifier__num_leaves': 20, 'classifier__reg_alpha': 0.1, 'classifier__reg_lambda': 0.5}
Лучший ROC_AUC на кросс-валидации: 0.7165388723830282

Метрики на тестовой выборке (LightGBM):
Accuracy: 0.6598
F1-score: 0.6633
ROC AUC: 0.6981

Classification Report (LightGBM):
              precision    recall  f1-score   support

           0       0.66      0.65      0.66        97
           1       0.66      0.67      0.66        97

    accuracy                           0.66       194
   macro avg       0.66      0.66      0.66       194
weighted avg       0.66      0.66      0.66       194



In [8]:
# Модель 4: XGBoost Классификатор
print("\n===== XGBoost Классификатор для SI (классификация) =====")
pipeline_xgb_class_si = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

param_grid_xgb_class_si = {
    'classifier__n_estimators': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__max_depth': [3, 5],
    'classifier__subsample': [0.7, 1.0],
    'classifier__colsample_bytree': [0.7, 1.0]
}

grid_search_xgb_class_si = GridSearchCV(
    pipeline_xgb_class_si,
    param_grid_xgb_class_si,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_xgb_class_si.fit(X_train_si_class, y_train_si_class)

print("Лучшие параметры для XGBoost:", grid_search_xgb_class_si.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_xgb_class_si.best_score_)

# Оценка на тестовой выборке
y_pred_xgb_class_si = grid_search_xgb_class_si.predict(X_test_si_class)
y_proba_xgb_class_si = grid_search_xgb_class_si.predict_proba(X_test_si_class)[:, 1]

accuracy_xgb_class_si = accuracy_score(y_test_si_class, y_pred_xgb_class_si)
f1_xgb_class_si = f1_score(y_test_si_class, y_pred_xgb_class_si)
roc_auc_xgb_class_si = roc_auc_score(y_test_si_class, y_proba_xgb_class_si)

print("\nМетрики на тестовой выборке (XGBoost):")
print(f"Accuracy: {accuracy_xgb_class_si:.4f}")
print(f"F1-score: {f1_xgb_class_si:.4f}")
print(f"ROC AUC: {roc_auc_xgb_class_si:.4f}")
print("\nClassification Report (XGBoost):")
print(classification_report(y_test_si_class, y_pred_xgb_class_si))


===== XGBoost Классификатор для SI (классификация) =====
Fitting 5 folds for each of 48 candidates, totalling 240 fits


C:\Users\admin\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [18:30:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Лучшие параметры для XGBoost: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__subsample': 0.7}
Лучший ROC_AUC на кросс-валидации: 0.7198414139972582

Метрики на тестовой выборке (XGBoost):
Accuracy: 0.6649
F1-score: 0.6409
ROC AUC: 0.6947

Classification Report (XGBoost):
              precision    recall  f1-score   support

           0       0.65      0.73      0.69        97
           1       0.69      0.60      0.64        97

    accuracy                           0.66       194
   macro avg       0.67      0.66      0.66       194
weighted avg       0.67      0.66      0.66       194



In [9]:
# Модель 5: CatBoost Классификатор
print("\n===== CatBoost Классификатор для SI (классификация) =====")
pipeline_cat_class_si = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', CatBoostClassifier(random_state=42, verbose=0))
])

param_grid_cat_class_si = {
    'classifier__iterations': [100, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__depth': [4, 6],
    'classifier__l2_leaf_reg': [1, 3]
}

grid_search_cat_class_si = GridSearchCV(
    pipeline_cat_class_si,
    param_grid_cat_class_si,
    cv=5,
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_cat_class_si.fit(X_train_si_class, y_train_si_class)

print("Лучшие параметры для CatBoost:", grid_search_cat_class_si.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_cat_class_si.best_score_)

# Оценка на тестовой выборке
y_pred_cat_class_si = grid_search_cat_class_si.predict(X_test_si_class)
y_proba_cat_class_si = grid_search_cat_class_si.predict_proba(X_test_si_class)[:, 1]

accuracy_cat_class_si = accuracy_score(y_test_si_class, y_pred_cat_class_si)
f1_cat_class_si = f1_score(y_test_si_class, y_pred_cat_class_si)
roc_auc_cat_class_si = roc_auc_score(y_test_si_class, y_proba_cat_class_si)

print("\nМетрики на тестовой выборке (CatBoost):")
print(f"Accuracy: {accuracy_cat_class_si:.4f}")
print(f"F1-score: {f1_cat_class_si:.4f}")
print(f"ROC AUC: {roc_auc_cat_class_si:.4f}")
print("\nClassification Report (CatBoost):")
print(classification_report(y_test_si_class, y_pred_cat_class_si))


===== CatBoost Классификатор для SI (классификация) =====
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Лучшие параметры для CatBoost: {'classifier__depth': 6, 'classifier__iterations': 200, 'classifier__l2_leaf_reg': 3, 'classifier__learning_rate': 0.01}
Лучший ROC_AUC на кросс-валидации: 0.7161797077381493

Метрики на тестовой выборке (CatBoost):
Accuracy: 0.6546
F1-score: 0.6257
ROC AUC: 0.7152

Classification Report (CatBoost):
              precision    recall  f1-score   support

           0       0.63      0.73      0.68        97
           1       0.68      0.58      0.63        97

    accuracy                           0.65       194
   macro avg       0.66      0.65      0.65       194
weighted avg       0.66      0.65      0.65       194



In [10]:
# Модель 6: Простая Нейронная Сеть (Keras Sequential)
print("\n===== Простая Нейронная Сеть для SI (классификация) =====")

# Функция для создания Keras-модели 
def build_nn_model_class(meta, hidden_layers=1, neurons=32, activation='relu',
                         optimizer='adam', learning_rate=0.001):
    n_features = meta["n_features_in_"]
    
    model = keras.Sequential()
    model.add(L.Input(shape=(n_features,)))
    
    for _ in range(hidden_layers):
        model.add(L.Dense(neurons, activation=activation))
        
    model.add(L.Dense(1, activation='sigmoid')) # Sigmoid для бинарной классификации
    
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

pipeline_nn_class_si = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', KerasClassifier(
        model=build_nn_model_class,
        hidden_layers=1,
        neurons=32,
        activation='relu',
        optimizer='adam',
        learning_rate=0.001,
        batch_size=32,
        epochs=50,
        verbose=0,
        random_state=42,
        loss='binary_crossentropy' # Лосс для бинарной классификации
    ))
])

param_grid_nn_class_si = {
    'classifier__hidden_layers': [1, 2],
    'classifier__neurons': [32, 64],
    'classifier__activation': ['relu'],
    'classifier__optimizer': ['adam'],
    'classifier__learning_rate': [0.001, 0.01],
    'classifier__batch_size': [32, 64],
    'classifier__epochs': [50, 100]
}

grid_search_nn_class_si = GridSearchCV(
    pipeline_nn_class_si,
    param_grid_nn_class_si,
    cv=3, # Уменьшаем CV для скорости NN
    scoring=scoring_metrics_class,
    refit='ROC_AUC',
    n_jobs=-1,
    verbose=1
)

grid_search_nn_class_si.fit(X_train_si_class, y_train_si_class)

print("Лучшие параметры для Нейронной Сети:", grid_search_nn_class_si.best_params_)
print("Лучший ROC_AUC на кросс-валидации:", grid_search_nn_class_si.best_score_)

# Оценка на тестовой выборке
y_pred_nn_class_si = grid_search_nn_class_si.predict(X_test_si_class)
y_proba_nn_class_si = grid_search_nn_class_si.predict_proba(X_test_si_class)[:, 1]

accuracy_nn_class_si = accuracy_score(y_test_si_class, y_pred_nn_class_si)
f1_nn_class_si = f1_score(y_test_si_class, y_pred_nn_class_si)
roc_auc_nn_class_si = roc_auc_score(y_test_si_class, y_proba_nn_class_si)

print("\nМетрики на тестовой выборке (Нейронная Сеть):")
print(f"Accuracy: {accuracy_nn_class_si:.4f}")
print(f"F1-score: {f1_nn_class_si:.4f}")
print(f"ROC AUC: {roc_auc_nn_class_si:.4f}")
print("\nClassification Report (Нейронная Сеть):")
print(classification_report(y_test_si_class, y_pred_nn_class_si))


===== Простая Нейронная Сеть для SI (классификация) =====
Fitting 3 folds for each of 32 candidates, totalling 96 fits


C:\Users\admin\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Лучшие параметры для Нейронной Сети: {'classifier__activation': 'relu', 'classifier__batch_size': 64, 'classifier__epochs': 50, 'classifier__hidden_layers': 1, 'classifier__learning_rate': 0.001, 'classifier__neurons': 32, 'classifier__optimizer': 'adam'}
Лучший ROC_AUC на кросс-валидации: 0.6953302617280612

Метрики на тестовой выборке (Нейронная Сеть):
Accuracy: 0.6443
F1-score: 0.6387
ROC AUC: 0.7062

Classification Report (Нейронная Сеть):
              precision    recall  f1-score   support

           0       0.64      0.66      0.65        97
           1       0.65      0.63      0.64        97

    accuracy                           0.64       194
   macro avg       0.64      0.64      0.64       194
weighted avg       0.64      0.64      0.64       194



Медианное значение log_SI составляет 0.5940. Распределение классов в целевой переменной si_above_median хорошо сбалансировано (484 для класса 0 и 483 для класса 1). Размеры обучающей и тестовой выборок, а также распределение классов в них, также являются сбалансированными.

1. Логистическая Регрессия
Лучшие параметры: {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Лучший ROC AUC на кросс-валидации: 0.6741
Метрики на тестовой выборке:
Accuracy: 0.6701
F1-score: 0.6484
ROC AUC: 0.7272
Classification Report:
Precision (класс 0 / класс 1): 0.65 / 0.69
Recall (класс 0 / класс 1): 0.73 / 0.61
F1-score (класс 0 / класс 1): 0.69 / 0.65
Модель логистической регрессии показала наиболее высокий ROC AUC (0.7272) среди всех протестированных моделей на тестовой выборке. Accuracy составила 0.6701, а F1-score — 0.6484. В отчете по классификации заметно, что для класса 0 (значение SI ниже медианы) Recall выше (0.73), чем для класса 1 (0.61), в то время как Precision для класса 1 (0.69) выше, чем для класса 0 (0.65). Это указывает на лучшую способность модели выявлять образцы, не превышающие медиану, но с некоторым количеством ложных срабатываний для класса 1.

2. Random Forest Классификатор
Лучшие параметры: {'classifier__max_features': 1.0, 'classifier__min_samples_leaf': 5, 'classifier__n_estimators': 200}
Лучший ROC AUC на кросс-валидации: 0.7222
Метрики на тестовой выборке:
Accuracy: 0.6237
F1-score: 0.6138
ROC AUC: 0.6771
Classification Report:
Precision (класс 0 / класс 1): 0.62 / 0.63
Recall (класс 0 / класс 1): 0.65 / 0.60
F1-score (класс 0 / класс 1): 0.63 / 0.61
Модель Random Forest продемонстрировала ROC AUC 0.6771 на тестовой выборке, что является одним из самых низких показателей в этом сравнении. Accuracy составила 0.6237, F1-score — 0.6138. Производительность модели на тестовой выборке заметно ниже, чем на кросс-валидации, что может указывать на некоторое переобучение. Precision и Recall для обоих классов относительно сбалансированы, но находятся на низком уровне.

3. LightGBM Классификатор
Лучшие параметры: {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100, 'classifier__num_leaves': 20, 'classifier__reg_alpha': 0.1, 'classifier__reg_lambda': 0.5}
Лучший ROC AUC на кросс-валидации: 0.7165
Метрики на тестовой выборке:
Accuracy: 0.6598
F1-score: 0.6633
ROC AUC: 0.6981
Classification Report:
Precision (класс 0 / класс 1): 0.66 / 0.66
Recall (класс 0 / класс 1): 0.65 / 0.67
F1-score (класс 0 / класс 1): 0.66 / 0.66
LightGBM показал ROC AUC 0.6981 на тестовой выборке. Accuracy и F1-score также были средними (0.6598 и 0.6633 соответственно). Precision и Recall для обоих классов очень хорошо сбалансированы (около 0.66-0.67), что указывает на равномерную производительность модели для обоих классов, но общий уровень метрик остается невысоким.

4. XGBoost Классификатор
Лучшие параметры: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.01, 'classifier__max_depth': 5, 'classifier__n_estimators': 200, 'classifier__subsample': 0.7}
Лучший ROC AUC на кросс-валидации: 0.7198
Метрики на тестовой выборке:
Accuracy: 0.6649
F1-score: 0.6409
ROC AUC: 0.6947
Classification Report:
Precision (класс 0 / класс 1): 0.65 / 0.69
Recall (класс 0 / класс 1): 0.73 / 0.60
F1-score (класс 0 / класс 1): 0.69 / 0.64
XGBoost продемонстрировал ROC AUC 0.6947 на тестовой выборке. Accuracy (0.6649) и F1-score (0.6409) также находятся на среднем уровне. Отчет по классификации выявил аналогичную тенденцию, как и у логистической регрессии: более высокий Recall для класса 0 (0.73) и более высокий Precision для класса 1 (0.69), что указывает на склонность модели к идентификации образцов, не превышающих медиану.

5. CatBoost Классификатор
Лучшие параметры: {'classifier__depth': 6, 'classifier__iterations': 200, 'classifier__l2_leaf_reg': 3, 'classifier__learning_rate': 0.01}
Лучший ROC AUC на кросс-валидации: 0.7162
Метрики на тестовой выборке:
Accuracy: 0.6546
F1-score: 0.6257
ROC AUC: 0.7152
Classification Report:
Precision (класс 0 / класс 1): 0.63 / 0.68
Recall (класс 0 / класс 1): 0.73 / 0.58
F1-score (класс 0 / класс 1): 0.68 / 0.63
CatBoost показал ROC AUC 0.7152 на тестовой выборке. Accuracy (0.6546) и F1-score (0.6257) оказались средними. Модель, как и Логистическая регрессия и XGBoost, демонстрирует смещение в сторону лучшего Recall для класса 0 (0.73) и более высокой Precision для класса 1 (0.68), при этом Recall для класса 1 является самым низким среди всех моделей (0.58).

6. Простая Нейронная Сеть
Лучшие параметры: {'classifier__activation': 'relu', 'classifier__batch_size': 64, 'classifier__epochs': 50, 'classifier__hidden_layers': 1, 'classifier__learning_rate': 0.001, 'classifier__neurons': 32, 'classifier__optimizer': 'adam'}
Лучший ROC AUC на кросс-валидации: 0.6953
Метрики на тестовой выборке:
Accuracy: 0.6443
F1-score: 0.6387
ROC AUC: 0.7062
Classification Report:
Precision (класс 0 / класс 1): 0.64 / 0.65
Recall (класс 0 / класс 1): 0.66 / 0.63
F1-score (класс 0 / класс 1): 0.65 / 0.64
Простая нейронная сеть продемонстрировала ROC AUC 0.7062 на тестовой выборке. Accuracy (0.6443) и F1-score (0.6387) также были средними. Precision и Recall для обоих классов относительно сбалансированы, но на невысоком уровне.

### Общий вывод по классификации: SI выше медианы
Результаты классификации для SI выше медианы оказались значительно ниже, чем для IC50 и CC50. ROC AUC для большинства моделей находится в диапазоне 0.67–0.73, а Accuracy и F1-score не превышают 0.67. Это согласуется с ранее полученными слабыми результатами регрессии для log_SI и указывает на общую сложность прогнозирования этого показателя на основе текущего набора признаков. Наилучшие результаты по ROC AUC на тестовой выборке показала Логистическая Регрессия (0.7272), что является неожиданностью, так как для регрессии она была самой слабой моделью. Среди остальных моделей, LightGBM, XGBoost, CatBoost и Нейронная сеть показали ROC AUC в диапазоне 0.69-0.71, а Random Forest — самый низкий (0.6771).

Переобучение/Недообучение:
У некоторых моделей, таких как Random Forest, наблюдается заметное падение ROC AUC на тестовой выборке по сравнению с кросс-валидацией (например, у Random Forest с 0.7222 до 0.6771), что может указывать на склонность к переобучению на тренировочных данных.
Логистическая Регрессия показала себя неожиданно лучше остальных моделей по ROC AUC на тестовой выборке, хотя общий уровень производительности всех моделей остаётся умеренным. 